In [1]:
from Project1_Code import *

# Mathematics for Machine Learning

In [2]:
A = np.array([[1, 2, 3, 8, 9],
             [3, 5, 7, 10, 14],
             [1, 4, 8, 8, 8]])

U, S, Vh = np.linalg.svd(A)

A_k = get_low_rank(U, S, Vh, 2)


In [5]:
get_errors(A, U, S, Vh, norm = "nuc")

[3.23228721e+01 3.65203352e+00 3.07361127e-28]


array([3.23228721e+01, 3.65203352e+00, 3.07361127e-28])